In [ ]:
# ---------------------Vector Auto Regression ----------------------
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# VECTOR AUTO REGRESSION EQUATION:

In [ ]:
# .)In this like in sales it increses with date suppose there is another factor 
# production base on which the sales is incrseing so we hve to tke that facor also
# in consideration so in folrmula that factor previos state will also eb added .

In [ ]:
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/Raotbl6.csv'
macro_data = pd.read_csv(filepath, parse_dates=['date'], index_col='date')
print(macro_data.shape)  # (123, 8)
macro_data.head()

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = macro_data[macro_data.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(macro_data.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

In [ ]:
# .)For multivainte in CAR we are tking 2 columns from above 'rgnp' anf 'ulc'

# .)the lw checking data stationary or not of both columns by dickey fuler test. if 
# p value less than 0.05 then it stationary otherwise not.

ad_fuller_result_1 = adfuller(macro_data['rgnp'])

print('realgdp')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(macro_data['ulc'])

print('realcons')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

In [ ]:
# .)Above data is not stationary so i am applying differncing time shift. 

# acro_data['rgnp']: This selects the rgnp column from the macro_data DataFrame.
# .diff(): This method computes the difference between each element in the column and the previous element, effectively performing a first-order differencing.
# [1:]: This slices the differenced series to exclude the first element, which is NaN (because there is no previous element to subtract from the first element).
ad_fuller_result_1 = adfuller(macro_data['rgnp'].diff()[1:])

print('realgdp')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(macro_data['ulc'].diff()[1:])

print('realcons')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')
# .)Now both p values are less than 0.0.5 so the data is stationary.
# we hve to is multiple time diff until data become stationary.

In [ ]:
# .)When we deal with 2 timeseries columns we hve to check the correltion btw them thus we 
# can do by grangercausalitytests test.

# .)If p value is leas then 0.05 theat means it cause.


# The code granger_1 = grangercausalitytests(macro_data[['ulc', 'rgnp']], 4) is used to perform the Granger causality test, which is a statistical hypothesis test for determining whether one time series can predict another time series. Specifically, this code is testing whether the time series ulc (unit labor cost) can predict the time series rgnp (real gross national product) and vice versa.

# p-Value: Helps determine the statistical significance of the test. A lower p-value (typically less than 0.05) suggests that the lagged values of one series significantly improve the prediction of the other series.
print('rgnp causes ulc?\n')
print('------------------')
granger_1 = grangercausalitytests(macro_data[['ulc', 'rgnp']], 4)

print('\nulc causes rgnp?\n')
print('------------------')
granger_2 = grangercausalitytests(macro_data[['rgnp', 'ulc']], 4)

# Making new dataframe of selected column

In [ ]:
macro_data = macro_data[['ulc','rgnp']]
print(macro_data.shape)
     

In [ ]:
train_df=macro_data[:-12]
test_df=macro_data[-12:]

In [ ]:
print(test_df.shape)

In [ ]:
# .)The value were not stationary i hve apply diff of 1 to mke it stationary as i check 
# it above after mking diff of 1 it becomes stationary .
model = VAR(train_df.diff()[1:])

In [ ]:
# .)By select order it will give the value of AIC and many more and we hve to chekc 
# that at which index the AIV value is minimum at we will tke taht index an that will 
# be the value of lag which we hve to use for good model . 
sorted_order=model.select_order(maxlags=20)
print(sorted_order.summary())
# .)The blwe at 4 index minimum value is presenty. 

In [ ]:
# .)In blw i hvent pass statinary data i hve passed orignal data abd in order previously 3
# para which were p i and q no in VARMAX we giv only p vlu which are lags which we find
# above is 4 and i is 0 and enforce_stationarity= True will enforce the 
# parameter that it they should enforcestationay.

# .)The blw VARMAX make forecasting es thatswhy iam using it . 
# 4 is aut reg lags and 0 is moving avg lags.mov avg dont exist so its 0;
var_model = VARMAX(train_df, order=(4,0),enforce_stationarity= True)
fitted_model = var_model.fit(disp=False)
print(fitted_model.summary())

In [ ]:
n_forecast = 12
predict = fitted_model.get_prediction(start=len(train_df),end=len(train_df) + n_forecast-1)#start="1989-07-01",end='1999-01-01')

predictions=predict.predicted_mean

In [ ]:

predictions.columns=['ulc_predicted','rgnp_predicted']
predictions

In [ ]:
test_vs_pred=pd.concat([test_df,predictions],axis=1)

In [ ]:
test_vs_pred.tail()

In [ ]:
test_vs_pred.plot(figsize=(12,5))

In [ ]:
from sklearn.metrics import mean_squared_error
import math 
from statistics import mean

rmse_ulc=math.sqrt(mean_squared_error(predictions['ulc_predicted'],test_df['ulc']))
print('Mean value of ULC is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['ulc'])
                                                                         ,rmse_ulc))

rmse_rgnp=math.sqrt(mean_squared_error(predictions['rgnp_predicted'],test_df['rgnp']))

print('Mean value of RGNP is : {}. Root Mean Squared Error is :{}'.
      format(mean(test_df['rgnp']),rmse_rgnp))

In [ ]:
Ratio of RMSE to Mean for ULC:

Ratio for ULC
=
RMSE of ULC
Mean value of ULC
=
52.74258110587016
178.56666666666666


Ratio for ULC= 
Mean value of ULC
RMSE of ULC
​
 = 
178.56666666666666
52.74258110587016
​
 
 
Ratio of RMSE to Mean for RGNP:

Ratio for RGNP
=
RMSE of RGNP
Mean value of RGNP
=
342.36207564516775
3970.6916666666666


Ratio for RGNP= 
Mean value of RGNP
RMSE of RGNP
​
 = 
3970.6916666666666
342.36207564516775


​
 
Let's calculate these ratios:

Ratio for ULC
=
52.74258110587016
178.56666666666666
≈
0.2953
Ratio for ULC= 
178.56666666666666
52.74258110587016
​
 ≈0.2953
 
 

Ratio for RGNP
=
342.36207564516775
3970.6916666666666
≈
0.0862

Ratio for RGNP= 
3970.6916666666666
342.36207564516775
​
 ≈0.0862